# Intent Detection LoRA

Sample notebook utilizing the `DORIE` package to fine tune an insurance intent detection, classification, model. This serves as an example demonstrating how to utilize the library for downstream classification tasks. This sequence classification task focus on classifying customer utterances into a multi-class intent to help the contact center route the phone call/ chat through a pre-defined or dynamic route to satify the customers needs.

In [1]:
from pathlib import Path
import sys

sys.path.insert(0, str(Path().resolve().parent))

from libs.dorie.intent.finetune import Intent
from libs.dorie.loader.adaptation import return_peft_model
from libs.dorie.loader.datatokenizer import MyDataset

from peft import TaskType

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load the dataset
See [Synthetic data notebook](./Synthetic_Data_Generation.ipynb) for more information on generating data utilizing OpenAI API. The dataset will be loaded from HuggingFace Data Hub. You can also navigate to HuggingFace to see the [synthetic_insurance_data](https://huggingface.co/datasets/stevenloaiza/synthetic_insurance_data).

In [2]:
# Set Data configuration, the intent class will load the data from the path specified here
datapath= "stevenloaiza/synthetic_insurance_data"
dataclass = MyDataset(path=datapath)

# The data parameter will not be used explicitly, since it will be loader in the finetuning modules implicitly. 
# Loading it here is just to show the data that will be used for the finetuning process
mydata = dataclass.loader()
mydata

Map: 100%|██████████| 435/435 [00:00<00:00, 3226.38 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 785
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 435
    })
})

## Load PEFT LoRA model

 * This section of the code is responsible for loading a PEFT (Parameter-Efficient Fine-Tuning) LoRA (Low-Rank Adaptation) model.
 * PEFT LoRA models are used to fine-tune large language models efficiently by adapting a small number of parameters.
 * This approach is particularly useful for tasks like intent detection, where the model needs to be adapted to understand specific intents from user inputs.
 * The model loading process typically involves specifying the model architecture, loading pre-trained weights, and preparing the model for inference or further training.

In [3]:
BASE_MODEL = "roberta-base"
LORA_CONFIG = {
    "task_type": TaskType.SEQ_CLS, 
    "inference_mode": False, 
    "r": 8, 
    "lora_alpha": 32, 
    "lora_dropout": 0.1
}
lora_model = return_peft_model(
    model_name_or_path = f"{BASE_MODEL}", 
    lora_config=LORA_CONFIG,
    verbose=True
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 887,042 || all params: 125,534,212 || trainable%: 0.7066


## Set Up Intent Detection Class

In this section, we will set up the intent detection class using the `Intent` class from the `DORIE` package. We will configure the class with the dataset and the PEFT LoRA model that we have loaded in the previous steps. This setup will allow us to fine-tune the model for the intent detection task.


In [5]:
TRAINING_CONFIG = {   
    "baseModel": f"{BASE_MODEL}",
    "device": "cpu",
    "modelArgs": {
        "output_dir": "./results",
        "num_train_epochs": 1,
        "per_device_train_batch_size": 32,
        "per_device_eval_batch_size": 32,
        "eval_strategy": "epoch",
        "save_strategy": "epoch",
        "learning_rate": 5e-5,
        "save_total_limit": 2,
        "load_best_model_at_end": "true",
        "metric_for_best_model": "accuracy",
        "greater_is_better": "true",
        "save_on_each_node": "true"
    },
    "model": lora_model
}

intent_classifier = Intent(
    datapath = datapath,
    config = TRAINING_CONFIG,
    dataclass = dataclass,
    # Local model trainer
    trainer = None,
    inference_text = """I was in a car accident this morning on Clifford ave. This other person rearended me and 
    I need to file a claim but I don't know how to start the process."""
)

## Train
The `train` method of the `intent_classifier` object is called to start the training process. This method will fine-tune the model using the provided dataset and configuration settings.

In [6]:
intent_classifier.train()

Map: 100%|██████████| 435/435 [00:00<00:00, 3161.98 examples/s]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
                                                  
  2%|▏         | 50/2500 [00:55<39:16,  1.04it/s]

{'eval_loss': 0.3879985809326172, 'eval_accuracy': 0.8390804597701149, 'eval_runtime': 6.9956, 'eval_samples_per_second': 62.182, 'eval_steps_per_second': 1.001, 'epoch': 1.0}


                                                   
  4%|▍         | 100/2500 [02:04<37:52,  1.06it/s]

{'eval_loss': 0.5785103440284729, 'eval_accuracy': 0.8229885057471265, 'eval_runtime': 6.7631, 'eval_samples_per_second': 64.32, 'eval_steps_per_second': 1.035, 'epoch': 2.0}


                                                    
  6%|▌         | 150/2500 [03:08<36:57,  1.06it/s]

...

                                                     
100%|██████████| 2500/2500 [2:04:02<00:00,  1.03it/s]

{'eval_loss': 1.670653223991394, 'eval_accuracy': 0.8275862068965517, 'eval_runtime': 8.4914, 'eval_samples_per_second': 51.229, 'eval_steps_per_second': 0.824, 'epoch': 50.0}


100%|██████████| 2500/2500 [2:04:09<00:00,  1.03it/s]

{'train_runtime': 7449.0687, 'train_samples_per_second': 5.269, 'train_steps_per_second': 0.336, 'train_loss': 0.05342383605092764, 'epoch': 50.0}


100%|██████████| 2500/2500 [2:04:10<00:00,  2.98s/it]


# Load To Hub
This section demonstrates how to save the fine-tuned model to the Hugging Face Hub. This allows for easy sharing and deployment of the model. The `push_to_hub` method is used to upload the model.

In [7]:
# Save Model
intent_classifier.push_to_hub(model_name='stevenloaiza/dorie-intent-classifier')

model.safetensors: 100%|██████████| 499M/499M [00:24<00:00, 20.5MB/s] 
2025-01-19 18:59:47,423 - __name__ - INFO - Model saved to Hugging Face Hub as stevenloaiza/dorie-intent-classifier
